# March Madness Data Analysis Log

In [1]:
import pandas as pd

df = pd.read_csv(
        "https://raw.githubusercontent.com/IGARDS/RPLib/master/data/dataset_tool_datasets.tsv",sep='\t')

df_games = df.loc[(df['Type'] == 'Games') & (df['Dataset Name'].str.contains('March Madness'))]
df_games.set_index('Dataset ID',inplace=True)
df_games

,Dataset Name,Description,Type,Data provenance,Download links
Dataset ID,,,,,
1,March Madness 2002,"2002 regular season games, list of teams, and ...",Games,https://github.com/IGARDS/RPLib/blob/main/pipe...,https://raw.githubusercontent.com/IGARDS/march...
2,March Madness 2003,"2003 regular season games, list of teams, and ...",Games,https://github.com/IGARDS/RPLib/blob/main/pipe...,https://raw.githubusercontent.com/IGARDS/march...
3,March Madness 2003,"2004 regular season games, list of teams, and ...",Games,https://github.com/IGARDS/RPLib/blob/main/pipe...,https://raw.githubusercontent.com/IGARDS/march...
4,March Madness 2003,"2005 regular season games, list of teams, and ...",Games,https://github.com/IGARDS/RPLib/blob/main/pipe...,https://raw.githubusercontent.com/IGARDS/march...
5,March Madness 2003,"2006 regular season games, list of teams, and ...",Games,https://github.com/IGARDS/RPLib/blob/main/pipe...,https://raw.githubusercontent.com/IGARDS/march...
6,March Madness 2003,"2007 regular season games, list of teams, and ...",Games,https://github.com/IGARDS/RPLib/blob/main/pipe...,https://raw.githubusercontent.com/IGARDS/march...
7,March Madness 2003,"2008 regular season games, list of teams, and ...",Games,https://github.com/IGARDS/RPLib/blob/main/pipe...,https://raw.githubusercontent.com/IGARDS/march...
8,March Madness 2003,"2009 regular season games, list of teams, and ...",Games,https://github.com/IGARDS/RPLib/blob/main/pipe...,https://raw.githubusercontent.com/IGARDS/march...
9,March Madness 2003,"2010 regular season games, list of teams, and ...",Games,https://github.com/IGARDS/RPLib/blob/main/pipe...,https://raw.githubusercontent.com/IGARDS/march...


## D Matrix Creation

In [2]:
for ix in df_games.index:
    !python ../../pipelines/create_D_from_games.py {ix} {ix}

load_D_from_games
https://raw.githubusercontent.com/IGARDS/marchmadness_study/master/data/2002teams.txt
https://raw.githubusercontent.com/IGARDS/marchmadness_study/master/data/2002games.txt
https://raw.githubusercontent.com/IGARDS/marchmadness_study/master/data/2002MadnessTeams.txt
team2           Alabama  Alcorn_St  Arizona  ...  Wisconsin  Wyoming  Xavier
team1                                        ...                            
Alabama             NaN        NaN      NaN  ...        NaN      NaN     NaN
Alcorn_St           NaN        NaN      NaN  ...        NaN      NaN     NaN
Arizona             NaN        NaN      NaN  ...        NaN      NaN     NaN
Boston_College      NaN        NaN      NaN  ...        NaN      NaN     NaN
Boston_Univ         NaN        NaN      NaN  ...        NaN      NaN     NaN
...                 ...        ...      ...  ...        ...      ...     ...
Wake_Forest         NaN        NaN      0.0  ...        NaN      NaN     NaN
Winthrop            NaN 

## Manual Update to dataset_tool_Ds.tsv

Now you must manually make changes to dataset_tool_Ds.tsv and push.

## LOP Card Creation

### Dataset IDs

In [57]:
for ix in df_games.index:
    print(ix)
    !python ../../pipelines/create_lop_card.py {ix} {ix}

1
    Dataset ID                                               Link
0            1  https://raw.githubusercontent.com/IGARDS/RPLib...
1            2  https://raw.githubusercontent.com/IGARDS/RPLib...
2            3  https://raw.githubusercontent.com/IGARDS/RPLib...
3            4  https://raw.githubusercontent.com/IGARDS/RPLib...
4            5  https://raw.githubusercontent.com/IGARDS/RPLib...
5            6  https://raw.githubusercontent.com/IGARDS/RPLib...
6            7  https://raw.githubusercontent.com/IGARDS/RPLib...
7            8  https://raw.githubusercontent.com/IGARDS/RPLib...
8            9  https://raw.githubusercontent.com/IGARDS/RPLib...
9           10  https://raw.githubusercontent.com/IGARDS/RPLib...
10          11  https://raw.githubusercontent.com/IGARDS/RPLib...
11          12  https://raw.githubusercontent.com/IGARDS/RPLib...
(65, 65)
Academic license - for non-commercial use only - expires 2022-09-05
Using license file /home/jupyter-pander14/gurobi.lic
Found mult

## Show how you can import a LOP Card

In [58]:
import sys
from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. 
sys.path.insert(0,home)

In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
import RPLib.pyrplib as pyrplib

In [ ]:
lop_card = pyrplib.base.LOPCard.from_json("./1_lop_card.json")

In [62]:
lop_card._instance

D                    {'Alabama': {'Alabama': 0.0, 'Alcorn_St': 0.0,...
obj                                                              196.0
solutions            [[59, 58, 54, 45, 42, 56, 62, 49, 47, 46, 55, ...
max_tau_solutions                                                 None
centroid_x           [[0.0, 0.1104604688, 0.5539557967, 0.115475308...
outlier_solution     [40, 53, 22, 2, 38, 61, 5, 12, 26, 20, 50, 51,...
dataset_id                                                           1
source_dataset_id                                                    1
dtype: object

In [63]:
pd.DataFrame(lop_card.solutions)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,59,58,54,45,42,56,62,49,47,46,...,29,60,22,15,0,53,40,1,17,12
1,40,53,22,2,38,61,5,12,26,20,...,59,10,7,13,18,19,35,58,11,41
2,59,58,54,49,47,45,42,56,62,46,...,15,12,5,1,2,0,22,53,40,17


## Plotting examples

In [64]:
import ranking_toolbox.pyrankability as pyrankability

In [71]:
D = pd.DataFrame(lop_card.D)

In [72]:
x=pd.DataFrame(lop_card.centroid_x,index=D.index,columns=D.columns)
g,scores,ordered_xstar=pyrankability.plot.show_single_xstar(x)

In [73]:
g

alt.Chart(...)